# Text Summarization



Улучшим два метода: TextRank и Extractive RNN.

Датасет: gazeta.ru

Возможно, в датасете находятся пустые данные. Проверим эту гипотезу, и если понадобится, сделаем предобработку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [ ]:
!pip install razdel networkx pymorphy2 nltk rouge==0.3.1 summa 

In [3]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [4]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

В качестве метрик здесь и далее используем BLEU и [ROUGE](https://).<br><br>

* **ROUGE-N** – measures unigram, bigram, trigram and higher order n-gram overlap
* **ROUGE-L** – measures longest matching sequence of words using LCS. An advantage of using LCS is thatit does not require consecutive matches but in-sequence matches that reflect sentence level wordorder. Since it automatically includes longest in-sequence common n-grams, you don’t need apredefined n-gram length.

In [5]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
import razdel

def calc_lead_n_score(records, n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'f': 0.23804097238957525, 'p': 0.22208274285774904, 'r': 0.37762764047433917}, 'rouge-2': {'f': 0.10027796832321115, 'p': 0.09647636782929753, 'r': 0.15833772153385062}, 'rouge-l': {'f': 0.1835646488408507, 'p': 0.2022959168891477, 'r': 0.34937017731940756}}


# Extractive RNN

## Oracle summary

Для сведения задачи к extractive summarization мы должны выбрать те предложения из оригинального текста, которые наиболее похожи на наше целевое summary по нашим метрикам.

In [ ]:
import copy

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    
    score = -1.0
    summaries = []
    for _ in range(n_sentences):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [ ]:
from tqdm.notebook import tqdm
import razdel

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, 
                                                           calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)


calc_oracle_score(test_records)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. стоит отметить, что участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
BLEU:  0.531336150784986
ROUGE:  {'rouge-1': {'f': 0.36951810858804146, 'p': 0.4053281117404892, 'r': 0.3661389123393327}, 'rouge-2': {'f': 0.2087846693590

## Extractive RNN

In [ ]:
!pip install transformers
!pip install sentence_transformers

Теперь пробуем предсказать oracle summary

### BPE
Для начала сделаем BPE токенизацию

In [ ]:
!pip install youtokentome

In [8]:
import youtokentome as yttm

def train_bpe(records, model_path, model_type="bpe", vocab_size=10000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w") as temp:
        for record in records:
            text, summary = record['text'], record['summary']
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")

In [9]:
bpe_processor = yttm.BPE('BPE_model.bin')

### Словарь
Составим словарь для индексации токенов

In [10]:
vocabulary = bpe_processor.vocab()

### Кэш oracle summary
Закэшируем oracle summary, чтобы не пересчитывать их каждый раз

In [ ]:
from rouge import Rouge
import razdel
from tqdm.notebook import tqdm

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break
        text = record["text"]
        summary = record["summary"]

        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        record["sentences"] = sentences
        record["oracle_sentences"] = list(sentences_indicies)
        record["oracle_summary"] = oracle_summary

    return records[:nrows]

ext_train_records = add_oracle_summary_to_records(train_records, nrows=2048)
ext_val_records = add_oracle_summary_to_records(val_records, nrows=256)
ext_test_records = add_oracle_summary_to_records(test_records, nrows=256)

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pickle

with open ('/content/drive/My Drive/ext_records_.dat', 'wb') as f_out:
  pickle.dump(ext_train_records, f_out)
  pickle.dump(ext_val_records, f_out)
  pickle.dump(ext_test_records, f_out)

In [13]:
import pickle

with open ('/content/drive/My Drive/ext_records_full.dat', 'rb') as f_in:
  ext_train_records = pickle.load(f_in)
  ext_val_records = pickle.load(f_in)
  ext_test_records = pickle.load(f_in)

### Составление батчей

In [14]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


class BatchIterator():
    def __init__(self, records, vocabulary, batch_size, bpe_processor, 
                 shuffle=True, lower=True, max_sentences=30, 
                 max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = len(records)
        self.batch_size = batch_size
        self.bpe_processor = bpe_processor
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(self.num_samples / batch_size))
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]

            batch_inputs = []
            batch_outputs = []
            max_sentence_length = self.max_sentence_length # 0
            max_sentences = self.max_sentences # 0
            batch_records = []

            for data_ind in batch_indices:
                
                record = self.records[data_ind]
                batch_records.append(record)
                text = record["text"]
                summary = record["summary"]
                summary = summary.lower() if self.lower else summary

                if "sentences" not in record:
                    sentences = [sentence.text.lower() if self.lower else sentence.text for sentence in razdel.sentenize(text)][:self.max_sentences] # self.max_sentences
                else:
                    sentences = record["sentences"]
                max_sentences = max(len(sentences), max_sentences)
                
                # номера предложений, которые в нашем саммари
                if "oracle_sentences" not in record:
                    calc_score = lambda x, y: calc_single_score(x, y, self.rouge)
                    sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=calc_score, lower=self.lower, max_sentences=self.max_sentences)[1] # self.max_sentences
                else:
                    sentences_indicies = record["oracle_sentences"]
                
                # inputs - индексы слов в предложении
                inputs = [bpe_processor.encode(sentence)[:self.max_sentence_length] for sentence in sentences] # self.max_sentence_length
                max_sentence_length = max(max_sentence_length, max([len(tokens) for tokens in inputs]))
                
                # получение метки класса предложения
                outputs = [int(i in sentences_indicies) for i in range(len(sentences))]
                batch_inputs.append(inputs)
                batch_outputs.append(outputs)

            tensor_inputs = torch.zeros((self.batch_size, max_sentences, max_sentence_length), dtype=torch.long, device=self.device)
            tensor_outputs = torch.zeros((self.batch_size, max_sentences), dtype=torch.float32, device=self.device)


            for i, inputs in enumerate(batch_inputs):
                for j, sentence_tokens in enumerate(inputs):
                    tensor_inputs[i][j][:len(sentence_tokens)] = torch.LongTensor(sentence_tokens)

            for i, outputs in enumerate(batch_outputs):
                tensor_outputs[i][:len(outputs)] = torch.LongTensor(outputs)

            yield {
                'inputs': tensor_inputs, 
                'outputs': tensor_outputs,
                'records': batch_records
            }

In [16]:
train_iterator = BatchIterator(ext_train_records, vocabulary, 32, bpe_processor, device=device)
val_iterator = BatchIterator(ext_val_records, vocabulary, 32, bpe_processor, device=device)
test_iterator = BatchIterator(ext_test_records, vocabulary, 32, bpe_processor, device=device)

## Extractor -  SummaRuNNer
 https://arxiv.org/pdf/1611.04230.pdf


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

def train_model(model, train_iterator, val_iterator, vocabulary, bpe_processor,
                epochs_count=1, loss_every_nsteps=16, lr=0.001, device_name="cuda"):
    
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))

    device = torch.device(device_name)
    model = model.to(device)

    total_loss = 0
    start_time = time.time()

    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.BCEWithLogitsLoss().to(device)

    for epoch in range(epochs_count):
        for step, batch in enumerate(train_iterator):

            model.train()
            logits = model(batch["inputs"]) # Прямой проход
            loss = loss_function(logits, batch["outputs"]) # Подсчёт ошибки

            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            
            total_loss += loss.item()
            if step % loss_every_nsteps == 0 and step != 0:
                val_total_loss = 0
                val_batch_count = 0

                model.eval()
                for _, val_batch in enumerate(val_iterator):
                    logits = model(val_batch["inputs"]) # Прямой проход
                    val_total_loss += loss_function(logits, batch["outputs"]) # Подсчёт ошибки
                    val_batch_count += 1

                avg_val_loss = val_total_loss/val_batch_count
                print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
                total_loss = 0
                start_time = time.time()

        total_loss = 0
        start_time = time.time()

In [16]:
from sentence_transformers import SentenceTransformer

sentence_transformer_model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking', device)

100%|██████████| 501M/501M [00:27<00:00, 18.0MB/s]


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self): 
        super().__init__()

        self.encoder = sentence_transformer_model

    def forward(self, inputs, hidden=None):
        
        sentences_embeddings = self.encoder.encode(inputs.tolist(),  
                                                   batch_size=32, 
                                                   device=device, 
                                                   is_pretokenized=True, 
                                                   convert_to_tensor=True).to(device)

        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 sentence_encoder_hidden_size=768, 
                 hidden_size=256,
                 bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        
        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN()
                                                  
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):

        # [batch_size, seq num, seq_len]
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count) 
        # [batch_size * seq num, seq_len]

        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        # [batch_size, seq num, hidden_size]

        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        # [batch_size, seq num, hidden_size]

        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        # [batch_size, hidden_size]

        # W * h^T
        content = self.content_linear_layer(outputs).squeeze(2) # 1-representation
        # [batch_size, seq num]

        # h^T * W * d
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2) # 2-representation

        # [batch_size, seq num, hidden_size] * [batch_size, hidden_size, 1] = [batch_size, seq num, 1]
        return content + salience

$P\left(y_{j} = 1 \mid \mathbf{h}_{j}, \mathbf{s}_{j}, \mathbf{d}\right)=\sigma\left(W_{c} \mathbf{h}_{j} + \mathbf{h}_{j}^{T} W_{s} \mathbf{d}\right)$
--------------------

In [ ]:
model = SentenceTaggerRNN()
train_model(model, train_iterator, val_iterator, vocabulary, bpe_processor, device_name="cuda")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Trainable params: 135785473
Epoch = 0, Avg Train Loss = 0.3115, Avg val loss = 0.2150, Time = 40.35s
Epoch = 0, Avg Train Loss = 0.2489, Avg val loss = 0.2361, Time = 37.87s
Epoch = 0, Avg Train Loss = 0.2391, Avg val loss = 0.2306, Time = 38.18s


In [ ]:
import re
def punct_detokenize(text):
    text = text.strip()
    punctuation = ",.!?:;%"
    closing_punctuation = ")]}"
    opening_punctuation = "([}"
    for ch in punctuation + closing_punctuation:
        text = text.replace(" " + ch, ch)
    for ch in opening_punctuation:
        text = text.replace(ch + " ", ch)
    res = [r'"\s[^"]+\s"', r"'\s[^']+\s'"]
    for r in res:
        for f in re.findall(r, text, re.U):
            text = text.replace(f, f[0] + f[2:-2] + f[-1])
    text = text.replace("' s", "'s").replace(" 's", "'s")
    text = text.strip()
    return text


def postprocess(ref, hyp, is_multiple_ref=False, detokenize_after=False, tokenize_after=True):
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    return ref, hyp

def inference_summarunner(model, iterator, top_k=3):

    references = []
    predictions = []

    model.eval()
    for batch in test_iterator:

        logits = model(batch["inputs"])
        in_summary = torch.argsort(logits, dim=1)[:, -top_k:]
        
        for i in range(len(batch['outputs'])):

            summary = batch['records'][i]['summary']
            summary = summary.lower()
            predicted_summary = ' '.join([batch['records'][i]['sentences'][idx] for idx in in_summary[i].sort()[0]])

            summary, predicted_summary = postprocess(summary, predicted_summary)
   
            references.append(summary)
            predictions.append(predicted_summary)

    calc_scores(references, predictions)

In [ ]:
inference_summarunner(model, test_iterator, 3)

Count: 256
Ref: зарубежные спецслужбы намеренно ищут уязвимости в российском it-секторе , чтобы проводить масштабные кибератаки , заявил секретарь совбеза николай патрушев . по его словам , основные цели злоумышленников – объекты критической информационной инфраструктуры рф . эти атаки — за год несколько миллионов случаев — создают угрозу национальной безопасности .
Hyp: иностранные спецслужбы проводят целенаправленный поиск уязвимостей российского it-сектора , чтобы массированно его атаковать . об этом заявил секретарь совета безопасности рф николай патрушев , передает риа « новости » . « основными целями для оказания вредоносного воздействия остаются объекты критической информационной инфраструктуры россии , что создаст реальные угрозы национальной безопасности » , – подчеркнул он .
BLEU:  0.4483358093852308
ROUGE:  {'rouge-1': {'f': 0.31733181247536896, 'p': 0.3031341514395033, 'r': 0.353196756345429}, 'rouge-2': {'f': 0.14277322190268435, 'p': 0.134759321871149, 'r': 0.163397261827

# TextRank

In [ ]:
!pip install -Uq razdel   networkx  rouge==0.3.1 
!pip install -Uq transformers youtokentome
!pip install pymorphy2
!pip install sentence_transformers

In [17]:
import torch
import random
import pandas as pd
from itertools import combinations
import networkx as nx
import numpy as np
import pymorphy2
import razdel
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
test_records = pd.DataFrame(ext_test_records)
test_records.index = test_records.reset_index().index
test_records['sentences'] = test_records.text.apply(lambda text: [sentence.text for sentence in razdel.sentenize(text)])

corpus = []
for sentences in test_records.sentences:
  corpus.extend(sentences)

In [19]:
# длины предложений
seq_len = [len([token.text for token in razdel.tokenize(sentence)]) for sentence in corpus] 
# число предложений в текстах
seq_num = test_records.sentences.apply(lambda sentences: len(sentences)) 

In [20]:
def sort_corpus_by_len(corpus, seq_len):
    corpus_seq_len = list(zip(list(zip(corpus, range(len(corpus)))), seq_len))
    sorted_by_len_corpus = sorted(corpus_seq_len, key=lambda x: x[1])[::-1]
    sorted_sent_idx = [pair_sent_idx for pair_sent_idx, _ in sorted_by_len_corpus]
    sorted_sents = [sent for sent, _ in sorted_sent_idx]
    sorted_indices = [idx for _, idx in sorted_sent_idx]
    return sorted_sents, sorted_indices

def get_corpus_order_embeddings(embeddings, sorted_indices):
     embedding_order = sorted(list(zip(embeddings, sorted_indices)), key=lambda idx: idx[1])
     embeddings = [emb for emb, _ in embedding_order]
     return embeddings

In [21]:
def batch_gen(corpus, batch_size):
    num_batches = len(corpus) // batch_size + int(len(corpus) % batch_size == 0)

    for i in range(num_batches+1):
      yield corpus[i*batch_size:((i+1)*batch_size if i != num_batches else None)]

In [22]:
def get_docs_embeddings(model, corpus, iterator, batch_size, seq_num, seq_len, device, sorted=False):
    
    if sorted:
      corpus, sorted_indices = sort_corpus_by_len(corpus, seq_len)

    embeddings = []
    for sent_batch in iterator(corpus, batch_size):
        embeddings.extend(model.encode(sent_batch, batch_size=batch_size, device=device))
    
    if sorted:
      embeddings = get_corpus_order_embeddings(embeddings, sorted_indices)


    embeddings = [emb.reshape(1, -1) for emb in embeddings]
    array_embeddings = np.array(embeddings)

    
    docs_embeddings = []
    start, end = 0, 0
    for length in seq_num:
        start, end = end, end + length
        docs_embeddings.append(list(array_embeddings[start:end, :, :])) 

    return docs_embeddings

In [28]:
batch_size = 64
model = SentenceTransformer('embeddings_xlm_distilroberta_paraphrase', device)

100%|██████████| 501M/501M [00:20<00:00, 24.4MB/s]


In [29]:
embeddings = get_docs_embeddings(model, corpus, batch_gen, batch_size, seq_num, seq_len, device, sorted=True)

In [30]:
import pickle

with open ('/content/drive/My Drive/embeddings_xlm_distilroberta_paraphrase.dat', 'wb') as f_out:
  pickle.dump(embeddings, f_out)

In [35]:
import pickle

with open ('/content/drive/My Drive/embeddings_xlm_distilroberta_paraphrase.dat', 'rb') as f_in:
  embeddings = pickle.load(f_in)

In [36]:
test_records['embeddings'] = pd.Series(embeddings)

In [33]:
def my_sim(embedding1, embedding2, norm=cosine_similarity):

    return norm(embedding1 - embedding2)


def gen_text_rank_summary(text, embeddings, calc_similarity=my_sim, summary_part=0.1):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)] # список предложений в виде строк
    n_sentences = len(sentences)

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(embeddings[i], embeddings[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])

    return predicted_summary


def calc_text_rank_score(records, calc_similarity=my_sim, summary_part=0.1, nrows=1000):
    references = []
    predictions = []

    for text, summary, embeddings in records[['text', 'summary', 'embeddings']].values[:nrows]:

        references.append(summary)
        
        predicted_summary = gen_text_rank_summary(text, embeddings, calc_similarity, summary_part)
        
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

In [37]:
calc_text_rank_score(test_records, calc_similarity=my_sim, summary_part=0.08)

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: Православный телеканал «Спас», учредителем которого является Московская патриархия, запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер в Тверской области. «Здесь только Ты и Бог. Проживи месяц в Ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект.
BLEU:  0.34150026022819713
ROUGE:  {'rouge-1': {'f': 0.14349482123691146, 'p': 0.1564047351989802, 'r': 0.14319448479089308}, 'rouge-2': {'f': 0.041246771555016486, 'p': 0.04463343504430261, 'r': 0.04137008638679155}, 'r